<a href="https://colab.research.google.com/github/BiancadeFrancisco/Regressao_Linear_SeguroVida/blob/main/CASE_Regress%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
seguro = pd.read_excel("/content/insurance.xlsx")

seguro.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.560,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
seguro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341 entries, 0 to 1340
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   idade              1341 non-null   int64  
 1   sexo               1338 non-null   object 
 2   imc                1341 non-null   float64
 3   quantidade_filhos  1341 non-null   int64  
 4   fumante            1341 non-null   object 
 5   regiao             1341 non-null   object 
 6   custos_seguro      1341 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.5+ KB


In [ ]:
# EXCLUIR NULOS:

seguro = seguro.dropna()

In [ ]:
seguro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1340
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   idade              1338 non-null   int64  
 1   sexo               1338 non-null   object 
 2   imc                1338 non-null   float64
 3   quantidade_filhos  1338 non-null   int64  
 4   fumante            1338 non-null   object 
 5   regiao             1338 non-null   object 
 6   custos_seguro      1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 83.6+ KB


In [ ]:
# DESCREVER AS CATEGORIAS CATEGÓRICAS

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(seguro.sexo)
seguro.sexo = le.transform(seguro.sexo)

le.fit(seguro.fumante)
seguro.fumante = le.transform(seguro.fumante)

le.fit(seguro.regiao)
seguro.regiao = le.transform(seguro.regiao)

seguro = pd.DataFrame(seguro)

In [ ]:
seguro.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.560,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [ ]:
# COLOCAR DADOS MESMA ESCALA NUMÉRICA:

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
seguro_norm = pd.DataFrame(scaler.fit_transform(seguro), index=seguro.index, columns=seguro.columns)

seguro_norm.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,0.021739,0.0,0.321227,0.0,1.0,1.000000,0.251611
1,0.000000,1.0,0.479150,0.2,0.0,0.666667,0.009636
2,0.217391,1.0,0.473500,0.6,0.0,0.666667,0.053115
3,0.326087,1.0,0.181464,0.0,0.0,0.333333,0.333010
4,0.304348,1.0,0.347592,0.0,0.0,0.333333,0.043816


In [ ]:
# RETIRAR MODELO OLS, PARA VERIFICAR P-VALOR E OUTROS DADOS:

import statsmodels.formula.api as smf

function = "custos_seguro~idade+sexo+imc+quantidade_filhos+fumante+regiao"

model = smf.ols(formula=function, data=seguro_norm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          custos_seguro   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     668.1
Date:                Wed, 19 Jul 2023   Prob (F-statistic):               0.00
Time:                        13:02:39   Log-Likelihood:                 1230.4
No. Observations:                1338   AIC:                            -2447.
Df Residuals:                    1331   BIC:                            -2410.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0479      0.00

In [ ]:
# PERCEBE-SE QUE APENAS A VARIÁVEL SEXO, ESTÁ COM P-VALOR ACIMA DO 5% DE LIMITE, RETIRAR ELA DO MODELO:

function = "custos_seguro~idade+imc+quantidade_filhos+fumante+regiao"

model = smf.ols(formula=function, data=seguro_norm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          custos_seguro   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     802.2
Date:                Wed, 19 Jul 2023   Prob (F-statistic):               0.00
Time:                        13:02:44   Log-Likelihood:                 1230.3
No. Observations:                1338   AIC:                            -2449.
Df Residuals:                    1332   BIC:                            -2417.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0488      0.00

In [ ]:
# DIVIDIR NOSSA BASE DE DADOS EM X E Y:

x = seguro_norm[["idade", "imc", "quantidade_filhos", "fumante", "regiao"]]
y = seguro_norm[["custos_seguro"]]



In [ ]:
# TREINAR E TESTAR NOSSO MODELO DE REGRESSÃO LINEAR:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

lr = LinearRegression()

lr.fit(x_train, y_train)

LinearRegression()

LinearRegression()

In [ ]:
# VERIFICAR O R²:

from sklearn import metrics

r_sq = lr.score(x,y)
print("r²:", r_sq)


r²: 0.7505629738411864


In [ ]:
# VERIFICAR O MAE: ERRO MÉDIO ABSOLUTO, NA BASE DE TREINO

y_pred_train = lr.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

MAE: 0.06760649849946623


In [ ]:
# VERIFICAR O MAE: ERRO MÉDIO ABSOLUTO, NA BASE DE TESTE (para verificar se está performando cfme o treino)

y_pred_test = lr.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

MAE: 0.06275502946015028


In [ ]:
# PERCEBE-SE QUE NÃO TEMOS UM OVERFIT, QUE AMBOS MODELOS ESTÃO PERFORMANDO DE MANEIRA SIMILAR!

In [ ]:
# VERIFICAR PERFORMANCE COM MODELO DE MACHINE LEARNING:

# 1° Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(x_train, y_train)

<ipython-input-79-17919ea3caca>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train)


RandomForestRegressor()

In [ ]:
# VERIFICAR O R² e MAE desse modelo:

from sklearn import metrics

r_sq = rf.score(x,y)
print("r²:", r_sq)

y_pred_train = rf.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = rf.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))


r²: 0.9525751645280108
MAE: 0.0165187798449077
MAE: 0.04369447972497011


In [ ]:
# VERIFICAR PERFORMANCE COM MODELO DE MACHINE LEARNING:

# 2° AdaBoost

from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor()
ada.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor()

In [ ]:
# VERIFICAR O R² e MAE desse modelo:

from sklearn import metrics

r_sq = ada.score(x,y)
print("r²:", r_sq)

y_pred_train = ada.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = ada.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

r²: 0.8319219166396821
MAE: 0.06314078657247074
MAE: 0.06432121262421267


In [ ]:
# VERIFICAR PERFORMANCE COM MODELO DE MACHINE LEARNING:

# 3° Gradient Boosting

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor()

In [ ]:
# VERIFICAR O R² e MAE desse modelo:

from sklearn import metrics

r_sq = gbr.score(x,y)
print("r²:", r_sq)

y_pred_train = gbr.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = gbr.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

r²: 0.8972711102032167
MAE: 0.03346236453828395
MAE: 0.03930824448286613


In [ ]:
# LEVANDO EM CONSIDERAÇÕES OS PARÂMETROS DE PERFORMANCE ENTRE OS MODELOS: LinearRegressor, RandomForestRegressor, AdaBoost e Gradient Boosting, o melhor foi o Gradient Boosting.
# AGORA IREMOS TUNAR ESSE MODELO:

from sklearn.model_selection import GridSearchCV

parameters = {"max_depth": [5, 10, 20],
              "min_samples_leaf": [2, 3, 4],
              "min_samples_split": [2, 3],
              "n_estimators": [50, 100, 200]}

grid_search = GridSearchCV(gbr, parameters, scoring="r2", cv=2, n_jobs=-1) #o que quero melhorar é o r²



In [ ]:
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [2, 3],
                         'n_estimators': [50, 100, 200]},
             scoring='r2')

In [ ]:
# VERIFICANDO MELHORES PARÂMETROS QUE ELE PODE DAR:

print(grid_search.best_estimator_)
print(grid_search.best_params_)

GradientBoostingRegressor(max_depth=5, min_samples_leaf=4, min_samples_split=3,
                          n_estimators=50)
{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 3, 'n_estimators': 50}


In [ ]:
# APLICANDO OS PARâMETROS QUE O MODELO PODE MELHOR PERFORMAR E TERINAR NOVAMENTE O MODELO

parameters = {"max_depth": [5],
              "min_samples_leaf": [4],
              "min_samples_split": [3],
              "n_estimators": [50]}

grid_search = GridSearchCV(gbr, parameters, scoring="r2", cv=2, n_jobs=-1)

grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5], 'min_samples_leaf': [4],
                         'min_samples_split': [3], 'n_estimators': [50]},
             scoring='r2')

In [ ]:
best_model = grid_search.best_estimator_
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# TUNAR O MODELO COM OS MELHORES PARâMETROS:

gbr_tunned = GradientBoostingRegressor(alpha = 0.9,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init = None,
 learning_rate = 0.1,
 loss = 'squared_error',
 max_depth = 5,
 max_features = None,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 4,
 min_samples_split = 3,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 50,
 n_iter_no_change = None,
 random_state = None,
 subsample = 1.0,
 tol = 0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [ ]:
# TREINAR NOSSO X E Y PARA O MODELO GBR TUNADO:

gbr_tunned.fit(x_train, y_train)
gbr_tunned_pred = gbr_tunned.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# VERIFICAR O R² e MAE do modelo GBR Tunado:

r_sq = gbr_tunned.score(x,y)
print("r²:", r_sq)

y_pred_train = gbr_tunned.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = gbr_tunned.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

r²: 0.9039216612685981
MAE: 0.031497465191872846
MAE: 0.04119140817833048


In [ ]:
# OBJETIVO ALCANÇADO! conseguimos aumentar o r² tunando o modelo que performou melhor.